In [1]:
from keras.models import load_model
import os
import keras.backend.tensorflow_backend as tb
tb._SYMBOLIC_SCOPE.value = True
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from keras_facenet import FaceNet


import time
import json
import mysql.connector
from SocialProcessor import SocialProcessor

import vk_api
import threading

%matplotlib inline

Using TensorFlow backend.


In [2]:
class StoppableThread(threading.Thread):
    """
    Thread class with a stop() method. The thread itself has to check
    regularly for the stopped() condition.
    
    """
    func = None
    args = {}
    processed_num = 0

    def __init__(self,  *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()
    
    def run(self):
        self.args['thread'] = self
        self.func(**self.args)
    

In [3]:
model_path = 'keras-facenet-master/model/keras/model/facenet_keras.h5'
model = load_model(model_path)

/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def aunt_vk():
    return input(), True


def capt_vk(captcha):
    print(captcha.get_url())
    inp = input()
    captcha.try_again(inp)

In [5]:
credentials = {}
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

In [6]:
vk_session = vk_api.VkApi(credentials['login'], credentials['password'], auth_handler=aunt_vk, captcha_handler=capt_vk)
vk_session.auth()

api = vk_session.get_api()

In [7]:
def performVkProcessing(delay=0, count=1, thread=False):
    conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")
    processor = SocialProcessor(conn, model, prefix='/Users/aleksandrdremov/PycharmProjects/GlobalScraper/', batch=600, limit=500)
    time.sleep(delay)
    while count>0:
        if type(thread) != type(False):
            if thread.stopped():
                break
        try:
            task = processor.getTask()
        except:
            return
        user_id = task[0]
        query_id = task[-1]
        processor.delTask(query_id)
        
        processor.processVkUser(api, user_id, des_type="y")
        if type(thread) != type(False):
            thread.processed_num += 1
        count-=1

In [ ]:
performVkProcessing(0, 1)

In [8]:
threads = []
threading.active_count()

5

In [17]:
def addThreads(current_num, max_num=20):
    t_num = max(0, max_num - current_num)
    limits = 10
    step = 1.5
    params = [(i*step, limits) for i in range(t_num)]
    print("Added %s threads"% (len(params)))
    for args in params:
        x = StoppableThread()
        x.func = performVkProcessing
        args = {"delay": args[0], "count": args[1]}
        x.args = args
        x.start()
        threads.append(x)


356078419:
Added 0 rows
29373952:
Procesing 527759031


Empty face: https://sun9-20.userapi.com/c626523/v626523828/238f7/jWbQtNWmVj4.jpg (1700, 1564, 807, 605)



In [18]:
avg = 0
betta = 0.9

In [19]:
while True:
    processed_already = 0
    prev_time = time.time()

    for i in threads:
        processed_already += i.processed_num
    
    active = sum([i.isAlive() for i in threads])
    print("Active threads:", active)
    addThreads(active)
    active = sum([i.isAlive() for i in threads])
    print("Active threads:", active)
    time.sleep(30)
    
    processed_new = 0
    new_time = time.time()
    
    for i in threads:
        processed_new += i.processed_num
    
    delta_proc =  processed_new - processed_already
    delta_time = new_time - prev_time
    speed = delta_proc / delta_time
    
    avg = (1 - betta) * speed + betta * avg
    
    print("Speed: %s obj/sec (%s in %s seconds). AVG: %s" % (speed, delta_proc, delta_time, avg))
    i = 0
    while i < len(threads):
        if threads[i].isAlive():
            i += 1
        else:
            threads.pop(i)
    

Active threads: 10
Added 10 threads
Active threads: 20
Empty face: https://sun3-10.userapi.com/OIWNIZ6aAVGwzEzlguzf7pxDJP3Uh499xKHkoA/XmicWNaIG3A.jpg (1518, 1631, 646, 807)
Procesing 142424778


Procesing 89532018


527759031:
Added 2 rows
348620732:
Added 9 rows
Procesing 72518720


Procesing 274670920


Procesing 454410948


...


1441477:
Added 0 rows
80039522:
Added 1 rows

Procesing 182217265


Procesing 3963926


Added 253 rows
Procesing 18096108


Empty face: https://sun9-52.userapi.com/c621923/v621923333/42bd7/soHp2iyr36A.jpg (139, 152, 56, 75)
Empty face: https://sun9-52.userapi.com/c621923/v621923333/42bd7/soHp2iyr36A.jpg (143, 221, 56, 75)
Speed: 0.39929022120683816 obj/sec (12 in 30.053328037261963 seconds). AVG: 0.13285355928789408
Active threads: 20
Added 0 threads
Active threads: 20
Empty face: https://sun9-63.userapi.com/c46/v46926/645/TkZt7WCOZX4.jpg (150, 139, 75, 57)
Empty face: https://sun9-63.userapi.com/c46/v46926/645/TkZt7WCOZX4.jpg (149, 73, 75, 57)
Empty face: https://sun9-63.userapi.com/c46/v46926/645/TkZt7WCOZX4.jpg (87, 137, 75, 57)
Speed: 0.0 obj/sec (0 in 30.053650856018066 seconds). AVG: 0.11956820335910467
Active threads: 20
Added 0 threads
Active threads: 20
Empty face: https://sun9-8.userapi.com/c840428/v840428760/1289d/ItpPoX3DFr0.jpg (1905, 1202, 640, 480)
Empty face: https://sun9-8.userapi.com/c840428/v840428760/1289d/ItpPoX3DFr0.jpg (1373, 1190, 640, 480)
Empty face: https://sun9-8.userapi.com/c840

KeyboardInterrupt: 

In [29]:
i = 0
while i < len(threads):
        if threads[i].isAlive():
            i += 1
        else:
            threads.pop(i)
sum([i.isAlive() for i in threads])

0

In [43]:
# conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")
# processor = SocialProcessor(conn, model, prefix='/Users/aleksandrdremov/PycharmProjects/GlobalScraper/', batch=512)
# processor.processVkUser(api, 1422)

In [20]:
[i.stop() for i in threads]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]